# dynamo db - create table

In [1]:
import os
import json
import time
import boto3
import pprint
import datetime
import numpy as np
import pandas as pd
import botocore.session
from boto3.dynamodb.conditions import Key, Attr

with open("../creds.json", "r") as f:
    creds = json.load(f)
    f.close()
    
print("imported modules.")

imported modules.


## dynamodb client

In [2]:
client = boto3.client('dynamodb',
  endpoint_url = "http://{}:{}".format(creds["dynamo_host"], creds["dynamo_port"]),
  aws_access_key_id=creds["aws-access-key"],
  aws_secret_access_key=creds["aws-secret-access-key"],
  region_name='us-east-2')

dynamodb = boto3.resource('dynamodb', \
            endpoint_url = "http://{}:{}".format(creds["dynamo_host"], creds["dynamo_port"]), \
            region_name='us-east-2')

print("created dynamo client.")

created dynamo client.


## create dynamo table

In [3]:
table = dynamodb.create_table(
    TableName='tweets',
    KeySchema=[
        {
            'AttributeName': 'user',
            'KeyType': 'HASH'  #Partition key
        },
        {
            'AttributeName': 'created_at',
            'KeyType': 'RANGE'  #Sort key
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'user',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'created_at',
            'AttributeType': 'S'
        }

    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 10,
        'WriteCapacityUnits': 10
    }
)

print("Table status:", table.table_status)

Table status: ACTIVE


## wait for table creation

In [4]:
session = botocore.session.get_session()
dynamodb_session = session.create_client('dynamodb', \
                    region_name='us-east-2',
                    endpoint_url = "http://{}:{}".format(creds["dynamo_host"], creds["dynamo_port"]),
                    aws_access_key_id=creds["aws-access-key"],
                    aws_secret_access_key=creds["aws-secret-access-key"]) # low-level client

waiter = dynamodb_session.get_waiter('table_exists')
waiter.wait(TableName="tweets")

WaiterError: Waiter TableExists failed: Max attempts exceeded. Previously accepted state: Matched expected service error code: ResourceNotFoundException

In [5]:
print("Table status:", table.table_status)

Table status: ACTIVE


## scan tweets table

In [6]:
tweets = dynamodb.Table("tweets")

response = tweets.scan()

In [7]:
response

{'Items': [],
 'Count': 0,
 'ScannedCount': 0,
 'ResponseMetadata': {'RequestId': 'a93ce2f1-46ae-49c6-a025-484f51f601b8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 05 Feb 2023 02:24:24 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '583912891',
   'x-amzn-requestid': 'a93ce2f1-46ae-49c6-a025-484f51f601b8',
   'content-length': '39',
   'server': 'Jetty(9.4.18.v20190429)'},
  'RetryAttempts': 0}}

In [8]:
! aws dynamodb list-tables --endpoint-url http://xanaxprincess.asuscomm.com:8000

{
    "TableNames": [
        "tweets"
    ]
}


In [9]:
! aws dynamodb describe-table --table-name "tweets" --endpoint-url http://xanaxprincess.asuscomm.com:8000

{
    "Table": {
        "AttributeDefinitions": [
            {
                "AttributeName": "user",
                "AttributeType": "S"
            },
            {
                "AttributeName": "created_at",
                "AttributeType": "S"
            }
        ],
        "TableName": "tweets",
        "KeySchema": [
            {
                "AttributeName": "user",
                "KeyType": "HASH"
            },
            {
                "AttributeName": "created_at",
                "KeyType": "RANGE"
            }
        ],
        "TableStatus": "ACTIVE",
        "CreationDateTime": 1675563321.991,
        "ProvisionedThroughput": {
            "LastIncreaseDateTime": 0.0,
            "LastDecreaseDateTime": 0.0,
            "NumberOfDecreasesToday": 0,
            "ReadCapacityUnits": 10,
            "WriteCapacityUnits": 10
        },
        "TableSizeBytes": 0,
        "ItemCount": 0,
        "TableArn": "arn:aws:dynamodb:ddblocal:000000000000:table/t